In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import probnum as pn

import linpde_gp

In [ ]:
import experiment_utils
from experiment_utils import config

config.experiment_name = "0000_cpu_stationary_1d"
config.target = "jmlr"
config.debug_mode = True

plt.rcParams.update(config.tueplots_bundle())

## Problem Definition

In [ ]:
import cpu

bvp = cpu.bvp_1D

In [ ]:
%matplotlib inline

import matplotlib.axes
import matplotlib.ticker

from probnum.typing import ArrayLike

from linpde_gp.typing import RandomProcessLike, RandomVariableLike


plt_grid = bvp.domain.uniform_grid(200)


def plot_belief(
    ax: matplotlib.axes.Axes,
    *,
    bvp: linpde_gp.problems.pde.BoundaryValueProblem = bvp,
    u: pn.randprocs.GaussianProcess | None = None,
    conditioned_on: list[str] = [],
    X_pde: ArrayLike | None = None,
    q_dot_A: RandomVariableLike | None = None,
    X_dts: ArrayLike | None = None,
    y_dts: ArrayLike | None = None,
    noise_dts: RandomVariableLike | None = None,
    show_u_label: bool = True,
):
    cond_events_str = ", ".join(_build_cond_events_str(conditioned_on))

    if len(cond_events_str) > 0:
        cond_events_str = fr"\mid {cond_events_str}"

    # Solution Belief
    if u is not None:
        u.plot(
            ax,
            plt_grid,
            num_samples=10,
            rng=np.random.default_rng(24),
            color="C0",
            label=fr"$\mathrm{{u}} {cond_events_str}$" if show_u_label else None,
        )
    
    # True Solution
    if bvp.solution is not None:
        bvp.solution.plot(
            ax,
            plt_grid,
            color="C1",
            label="$u^\star$" if show_u_label else None,
        )

    # PDE Observations
    if X_pde is not None:
        X_pde = np.asarray(X_pde)

        for i, x in enumerate(X_pde):
            ax.axvline(
                x,
                color="C3",
                alpha=0.1,
                linestyle="--",
                label=r"$\bm{X}_{\mathrm{PDE}}$" if i == 0 else None,
            )

    # Neumann Boundary Conditions
    if q_dot_A is not None:
        q_dot_A_label = (
            r"\dot{\mathrm{q}}_A"
            if isinstance(q_dot_A, pn.randvars.RandomVariable)
            else r"\dot{q}_A"
        )
        q_dot_A_cond_events_str = (
            cond_events_str
            if isinstance(q_dot_A, pn.randvars.RandomVariable)
            else ""
        )

        X_bc = np.asarray(bvp.domain.boundary)
        q_dot_A = pn.randvars.asrandvar(q_dot_A)

        slopes = [-q_dot_A[0] / cpu.kappa, q_dot_A[1] / cpu.kappa]

        linpde_gp.utils.plotting.plot_local_taylor_processes(
            ax,
            xs=X_bc,
            coeffs_xs=[
                pn.randvars.Normal(
                    np.array([u.mean(x), slope.mean]),
                    np.diag([0.0, slope.var]),
                )
                for x, slope in zip(bvp.domain, slopes)
            ],
            radius=0.3,
            color="C5",
            rel_fill_alpha=0.25,
            label=(
                f"${q_dot_A_label} {q_dot_A_cond_events_str}$"
            ),
        )

    # Measurements
    if X_dts is not None and y_dts is not None and noise_dts is not None:
        X_dts = np.asarray(X_dts)
        y_dts = np.asarray(y_dts)
        noise_dts = pn.asrandvar(noise_dts)

        ax.errorbar(
            X_dts,
            y_dts + noise_dts.mean,
            yerr=1.96 * noise_dts.std,
            fmt="+",
            capsize=2,
            color="C4",
            label=r"$(\bm{X}_{\mathrm{DTS}}, \bm{y}_{\mathrm{DTS}})$",
        )

    cpu.adjust_xaxis(ax)
    cpu.adjust_tempaxis(ax.yaxis)

    # ax.set_ylabel(
    #     r"Temperature (\unit{\degreeCelsius})",
    #     ha="left",
    #     y=1,
    #     rotation=0,
    #     labelpad=0,
    # )

    ax.legend()


def plot_pred_belief(
    ax,
    bvp: linpde_gp.problems.pde.BoundaryValueProblem = bvp,
    u: pn.randprocs.GaussianProcess | None = None,
    q_dot_V: RandomProcessLike | None = None,
    conditioned_on: list[str] = [],
    X_pde: ArrayLike | None = None,
    show_u_f_label: bool = True,
):
    cond_events_str = ", ".join(_build_cond_events_str(conditioned_on))

    if len(cond_events_str) > 0:
        cond_events_str = fr"\mid {cond_events_str}"

    # RHS Belief
    q_dot_V_label = r"\dot{" + (
        "q" if q_dot_V is None else r"\mathrm{q}"
    ) + r"}_V"

    if q_dot_V is not None:
        q_dot_V = linpde_gp.randprocs.asrandproc(q_dot_V)

        q_dot_V.plot(
            ax,
            plt_grid,
            num_samples=10,
            rng=np.random.default_rng(24),
            color="C2",
            label=fr"${q_dot_V_label} {cond_events_str}$",
        )

    # Differential Operator Image Belief
    if u is not None:
        bvp.pde.diffop(u).plot(
            ax,
            plt_grid,
            num_samples=10,
            rng=np.random.default_rng(24),
            color="C0",
            label=fr"$-\kappa \Delta \mathrm{{u}} {cond_events_str}$" if show_u_f_label else None,
        )
    
    # True Right-Hand Side
    bvp.pde.rhs.plot(
        ax,
        plt_grid,
        color="C1",
        label=r"$\dot{q}_V" + (
            "$"
            if q_dot_V is None
            else r"^\star$"
        ) if show_u_f_label else None,
    )
    
    # PDE Observations
    if X_pde is not None:
        if u is not None:
            X_pde = np.asarray(X_pde)
            y_pde = pn.randvars.asrandvar(
                bvp.pde.rhs(X_pde)
                if q_dot_V is None
                else q_dot_V(X_pde)
            )

            ax.errorbar(
                X_pde,
                y_pde.mean,
                yerr=1.96 * y_pde.std,
                fmt="+",
                capsize=2,
                c="C3",
                label=(
                    r"$(\bm{X}_{\mathrm{PDE}},"
                    f"{q_dot_V_label}"
                    r"(\bm{X}_{\mathrm{PDE}}))$"
                ),
            )
        else:
            X_pde = np.asarray(X_pde)

            for i, x in enumerate(X_pde):
                ax.axvline(
                    x,
                    color="C3",
                    alpha=0.1,
                    linestyle="--",
                    label=r"$\bm{X}_{\mathrm{PDE}}$" if i == 0 else None,
                )
    
    cpu.adjust_xaxis(ax)
    cpu.adjust_q_dot_V_axis(ax.yaxis)

    # ax.set_ylabel(
    #     r"Heat Flow (\unit{\watt\per\cubic\mm)}",
    #     ha="left",
    #     y=1,
    #     rotation=0,
    #     labelpad=0,
    # )

    ax.legend()


def _build_cond_events_str(
    conditioned_on: list[str]
) -> str:
    for key in conditioned_on:
        match key:
            case "pde":
                yield r"\mathrm{PDE}"
            case "dbc":
                yield r"\mathrm{DBC}"
            case "nbc":
                yield r"\mathrm{NBC}"
            case "dts":
                yield r"\mathrm{DTS}"
            case "stat":
                yield r"\mathrm{STAT}"
            case _:
                raise ValueError(f"Unknown event '{key}'")

## Simplified Model with Dirichlet Boundaries

### Prior

In [ ]:
u = pn.randprocs.GaussianProcess(
    mean=linpde_gp.functions.Constant(input_shape=(), value=57.0),
    cov=3.0 ** 2 * linpde_gp.randprocs.covfuncs.Matern(
        input_shape=(),
        nu=2.5,
        lengthscales=0.75 * cpu.width,
    ),
)

In [ ]:
plot_belief(
    ax=plt.gca(),
    bvp=cpu.bvp_1D_dbc,
    u=u,
)

In [ ]:
plot_pred_belief(
    ax=plt.gca(),
    bvp=cpu.bvp_1D_dbc,
    u=u,
)

In [ ]:
with plt.rc_context(config.tueplots_bundle(nrows=2, rel_width=0.5)):
    with plt.rc_context({
        "legend.facecolor": "white",
        "legend.framealpha": 0.8,
    }):
        fig, ax = plt.subplots(nrows=2, sharex=True)

        with plt.rc_context({"legend.loc": "lower left"}):
            plot_belief(
                ax=ax[0],
                bvp=cpu.bvp_1D_dbc,
                u=u,
            )

        with plt.rc_context({"legend.loc": "upper right"}):
            plot_pred_belief(
                ax=ax[1],
                bvp=cpu.bvp_1D_dbc,
                u=u,
            )

experiment_utils.savefig("01_simplified_dbc_00_prior")

In [ ]:
with plt.rc_context(config.tueplots_bundle(nrows=2, rel_width=0.5)):
    with plt.rc_context({
        "legend.facecolor": "white",
        "legend.framealpha": 0.8,
    }):
        fig, ax = plt.subplots(nrows=2, sharex=True)

        with plt.rc_context({"legend.loc": "lower left"}):
            plot_belief(
                ax=ax[0],
                bvp=cpu.bvp_1D_dbc,
                u=u,
            )

        with plt.rc_context({"legend.loc": "upper right"}):
            plot_pred_belief(
                ax=ax[1],
                bvp=cpu.bvp_1D_dbc,
            )

experiment_utils.savefig("01_simplified_dbc_00_prior_no_pred")

### Observations

In [ ]:
N_pde = 17

X_pde = bvp.domain.uniform_grid(N_pde, inset=0.03 * cpu.width)

### Posterior

#### Conditioning on the PDE

In [ ]:
u_cond_pde = u.condition_on_observations(
    Y=bvp.pde.rhs(X_pde),
    X=X_pde,
    L=bvp.pde.diffop,
)

In [ ]:
plot_belief(
    ax=plt.gca(),
    bvp=cpu.bvp_1D_dbc,
    u=u_cond_pde,
    conditioned_on=["pde"],
    X_pde=X_pde,
)

In [ ]:
plot_pred_belief(
    ax=plt.gca(),
    bvp=cpu.bvp_1D_dbc,
    u=u_cond_pde,
    conditioned_on=["pde"],
    X_pde=X_pde,
)

In [ ]:
with plt.rc_context(config.tueplots_bundle(nrows=2, rel_width=0.5)):
    with plt.rc_context({
        "legend.facecolor": "white",
        "legend.framealpha": 0.8,
    }):
        fig, ax = plt.subplots(nrows=2, sharex=True)

        with plt.rc_context({"legend.loc": "lower left"}):
            plot_belief(
                ax=ax[0],
                bvp=cpu.bvp_1D_dbc,
                u=u_cond_pde,
                conditioned_on=["pde"],
                X_pde=X_pde,
                show_u_label=False,
            )

        with plt.rc_context({"legend.loc": "upper right"}):
            plot_pred_belief(
                ax=ax[1],
                bvp=cpu.bvp_1D_dbc,
                u=u_cond_pde,
                conditioned_on=["pde"],
                X_pde=X_pde,
                show_u_f_label=False,
            )

experiment_utils.savefig("01_simplified_dbc_01_cond_pde")

#### Conditioning on the Dirichlet Boundary Conditions

In [ ]:
X_dbc, y_dbc = linpde_gp.problems.pde.get_1d_dirichlet_boundary_observations(
    cpu.bvp_1D_dbc.boundary_conditions
)

u_cond_pde_bc = u_cond_pde.condition_on_observations(
    Y=y_dbc,
    X=X_dbc,
)

In [ ]:
def plot_dbc(ax: matplotlib.axes.Axes, u_X_dbc: RandomVariableLike):
    X_dbc = np.asarray(bvp.domain.boundary)
    u_X_dbc = pn.randvars.asrandvar(u_X_dbc)

    ax.errorbar(
        X_dbc,
        u_X_dbc.mean,
        yerr=1.96 * u_X_dbc.std,
        fmt="+",
        capsize=2,
        color="C3",
        label=r"$(\bm{X}_\mathrm{BC}, u^\star(\bm{X}_\mathrm{BC}))$",
    )

In [ ]:
plot_belief(
    ax=plt.gca(),
    bvp=cpu.bvp_1D_dbc,
    u=u_cond_pde_bc,
    conditioned_on=["pde", "dbc"],
    X_pde=X_pde,
)

plot_dbc(plt.gca(), u_X_dbc=y_dbc)

plt.gca().legend()

In [ ]:
plot_pred_belief(
    ax=plt.gca(),
    bvp=cpu.bvp_1D_dbc,
    u=u_cond_pde_bc,
    conditioned_on=["pde", "dbc"],
    X_pde=X_pde,
)

In [ ]:
with plt.rc_context(config.tueplots_bundle(nrows=2, rel_width=0.5)):
    with plt.rc_context({
        "legend.facecolor": "white",
        "legend.framealpha": 0.8,
    }):
        fig, ax = plt.subplots(nrows=2, sharex=True)

        with plt.rc_context({"legend.loc": "lower left"}):
            plot_belief(
                ax=ax[0],
                bvp=cpu.bvp_1D_dbc,
                u=u_cond_pde_bc,
                conditioned_on=["pde", "dbc"],
                X_pde=X_pde,
                show_u_label=False,
            )

            plot_dbc(ax[0], u_X_dbc=y_dbc)

            ax[0].legend()

        with plt.rc_context({"legend.loc": "upper right"}):
            plot_pred_belief(
                ax=ax[1],
                bvp=cpu.bvp_1D_dbc,
                u=u_cond_pde_bc,
                conditioned_on=["pde", "dbc"],
                X_pde=X_pde,
                show_u_f_label=False,
            )
    
experiment_utils.savefig("01_simplified_dbc_02_cond_pde_dbc")

## Simplified Model with Neumann Boundary Conditions and Interior Measurements

### Prior

In [ ]:
u = pn.randprocs.GaussianProcess(
    mean=linpde_gp.functions.Constant(input_shape=(), value=57.0),
    cov=3.0 ** 2 * linpde_gp.randprocs.covfuncs.Matern(
        input_shape=(),
        nu=2.5,
        lengthscales=0.75 * cpu.width,
    ),
)

In [ ]:
with plt.rc_context(config.tueplots_bundle(rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower left"}):
        plot_belief(
            ax=plt.gca(),
            u=u,
        )

experiment_utils.savefig("02_simplified_nbc_00_prior_u")

In [ ]:
with plt.rc_context(config.tueplots_bundle(rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower left"}):
        plot_pred_belief(
            ax=plt.gca(),
            u=u,
        )

experiment_utils.savefig("02_simplified_nbc_00_prior_f")

In [ ]:
with plt.rc_context(config.tueplots_bundle(nrows=2, rel_width=0.44)):
    fig, ax = plt.subplots(nrows=2, sharex=True)

    # CPU Schematic
    cpu.plot_schematic(ax[0])

    ax[0].imshow(
        cpu.bvp_2D.pde.rhs(cpu.domain_2D.uniform_grid((200, 100))).T,
        cmap="coolwarm",
        norm=matplotlib.colors.TwoSlopeNorm(0.0),
        extent=[0.0, cpu.width, 0.0, cpu.height],
    )

    ax[0].plot([0.0, cpu.width], 2 * [cpu.core_centers_ys[1]], c="C1")

    # PDE RHS
    plot_pred_belief(ax[1])

    ax[1].legend(loc="upper right")

experiment_utils.savefig("02_simplified_nbc_00_geometry_rhs")

### Observations

In [ ]:
N_pde = 17

X_pde = bvp.domain.uniform_grid(N_pde, inset=0.03 * cpu.width)

In [ ]:
X_dts = cpu.X_dts_1D
y_dts = cpu.y_dts_1D
noise_dts = cpu.noise_dts_1D

### Posterior

#### Conditioning on the PDE

In [ ]:
u_cond_pde = u.condition_on_observations(
    Y=bvp.pde.rhs(X_pde),
    X=X_pde,
    L=bvp.pde.diffop,
)

In [ ]:
with plt.rc_context(config.tueplots_bundle(rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower left"}):
        plot_belief(
            ax=plt.gca(),
            u=u_cond_pde,
            conditioned_on=["pde"],
            X_pde=X_pde,
        )

experiment_utils.savefig("02_simplified_nbc_01_cond_pde_u")

In [ ]:
with plt.rc_context(config.tueplots_bundle(rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower center"}):
        plot_pred_belief(
            ax=plt.gca(),
            u=u_cond_pde,
            conditioned_on=["pde"],
            X_pde=X_pde,
        )

experiment_utils.savefig("02_simplified_nbc_01_cond_pde_f")

In [ ]:
with plt.rc_context(config.tueplots_bundle(nrows=2, rel_width=0.5)):
    with plt.rc_context({
        "legend.facecolor": "white",
        "legend.framealpha": 0.8,
    }):
        fig, ax = plt.subplots(nrows=2, sharex=True)

        with plt.rc_context({"legend.loc": "lower left"}):
            plot_belief(
                ax=ax[0],
                u=u_cond_pde,
                conditioned_on=["pde"],
                X_pde=X_pde,
                show_u_label=False,
            )

        with plt.rc_context({"legend.loc": "upper right"}):
            plot_pred_belief(
                ax=ax[1],
                u=u_cond_pde,
                conditioned_on=["pde"],
                X_pde=X_pde,
                show_u_f_label=False,
            )

experiment_utils.savefig("02_simplified_nbc_01_cond_pde")

#### Conditioning on Neumann Boundary Conditions

According to Fourier's law, the outward heat flux through the surface of the CPU is
given by $q(x) = \langle n(x), - k \nabla u(x) \rangle = - k \nabla_{n(x)} u(x)$.

In [ ]:
u_cond_pde_nbc = u_cond_pde

for idx, bcond in enumerate(bvp.boundary_conditions):
    u_cond_pde_nbc = u_cond_pde_nbc.condition_on_observations(
        Y=bcond.values.value,
        L=bcond.operator,
        X=bcond.boundary,
    )

In [ ]:
with plt.rc_context(config.tueplots_bundle(rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower left"}):
        plot_belief(
            ax=plt.gca(),
            u=u_cond_pde_nbc,
            conditioned_on=["pde", "nbc"],
            X_pde=X_pde,
            q_dot_A=cpu.q_dot_A_1D,
        )

experiment_utils.savefig("02_simplified_nbc_02_a_cond_pde_nbc_u")

In [ ]:
with plt.rc_context(config.tueplots_bundle(rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower center"}):
        plot_pred_belief(
            ax=plt.gca(),
            u=u_cond_pde_nbc,
            conditioned_on=["pde", "nbc"],
            X_pde=X_pde,
        )

experiment_utils.savefig("02_simplified_nbc_02_a_cond_pde_nbc_f")

In [ ]:
with plt.rc_context(config.tueplots_bundle(nrows=2, rel_width=0.5)):
    with plt.rc_context({
        "legend.facecolor": "white",
        "legend.framealpha": 0.8,
    }):
        fig, ax = plt.subplots(nrows=2, sharex=True)

        with plt.rc_context({"legend.loc": "lower left"}):
            plot_belief(
                ax=ax[0],
                u=u_cond_pde_nbc,
                conditioned_on=["pde", "nbc"],
                X_pde=X_pde,
                q_dot_A=cpu.q_dot_A_1D,
                show_u_label=False,
            )

        with plt.rc_context({"legend.loc": "upper right"}):
            plot_pred_belief(
                ax=ax[1],
                u=u_cond_pde_nbc,
                conditioned_on=["pde", "nbc"],
                X_pde=X_pde,
                show_u_f_label=False,
            )

experiment_utils.savefig("02_simplified_nbc_02_a_cond_pde_nbc")

#### Conditioning on Noisy Interior Measurements

In [ ]:
u_cond_pde_nbc_dts = u_cond_pde_nbc.condition_on_observations(
    Y=y_dts,
    X=X_dts,
    b=noise_dts,
)

In [ ]:
with plt.rc_context(config.tueplots_bundle(rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower left"}):
        plot_belief(
            ax=plt.gca(),
            u=u_cond_pde_nbc_dts,
            conditioned_on=["pde", "nbc", "dts"],
            X_pde=X_pde,
            q_dot_A=cpu.q_dot_A_1D,
            X_dts=X_dts,
            y_dts=y_dts,
            noise_dts=noise_dts,
        )

experiment_utils.savefig("02_simplified_nbc_03_a_cond_pde_nbc_dts_u")

In [ ]:
with plt.rc_context(config.tueplots_bundle(rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower center"}):
        plot_pred_belief(
            ax=plt.gca(),
            u=u_cond_pde_nbc_dts,
            conditioned_on=["pde", "nbc", "dts"],
            X_pde=X_pde,
        )

experiment_utils.savefig("02_simplified_nbc_03_a_cond_pde_nbc_dts_f")

In [ ]:
with plt.rc_context(config.tueplots_bundle(nrows=2, rel_width=0.5)):
    with plt.rc_context({
        "legend.facecolor": "white",
        "legend.framealpha": 0.8,
    }):
        fig, ax = plt.subplots(nrows=2, sharex=True)

        with plt.rc_context({"legend.loc": "lower left"}):
            plot_belief(
                ax=ax[0],
                u=u_cond_pde_nbc_dts,
                conditioned_on=["pde", "nbc", "dts"],
                X_pde=X_pde,
                q_dot_A=cpu.q_dot_A_1D,
                X_dts=X_dts,
                y_dts=y_dts,
                noise_dts=noise_dts,
                show_u_label=False,
            )

        with plt.rc_context({"legend.loc": "upper right"}):
            plot_pred_belief(
                ax=ax[1],
                u=u_cond_pde_nbc_dts,
                conditioned_on=["pde", "nbc", "dts"],
                X_pde=X_pde,
                show_u_f_label=False,
            )

experiment_utils.savefig("02_simplified_nbc_03_a_cond_pde_nbc_dts")

## Full Model

\begin{align*}
    \mathrm{u} & \sim \mathcal{GP}(m_u, k_u) \\
    \dot{\mathrm{q}}_V & \sim \mathcal{GP}(m_f, k_f) \\
    \dot{\mathrm{q}}_A & \sim \mathcal{GP}(m_g, k_g) \\
    \bm{\epsilon}_{\text{DTS}} & \sim \mathcal{N}(0, \sigma_{\text{DTS}}^2 \bm{I})
\end{align*}

\begin{align*}
    -\kappa \Delta \mathrm{u}(\bm{X}_{\text{PDE}}) - \dot{\mathrm{q}}_V(\bm{X}_{\text{PDE}}) & = 0 \\
    -\kappa \nabla_{\nu(\bm{X}_{\text{NBC}})} \mathrm{u}(\bm{X}_{\text{NBC}}) - \dot{\mathrm{q}}_{A}(\bm{X}_{\text{NBC}}) & = 0 \\
    \mathrm{u}(\bm{X}_{\text{DTS}}) + \bm{\epsilon}_{\text{DTS}} & = \bm{y}_{\text{DTS}} \\
    h_\text{CPU} \int_{\mathbb{D}} \dot{\mathrm{q}}_V(x) \mathrm{d}x + h_\text{CPU} \left( \dot{\mathrm{q}}_{A}(w_\text{CPU}) + \dot{\mathrm{q}}_{A}(0) \right) & = 0 \\
\end{align*}

### Prior

In [ ]:
ufg_prior = pn.randprocs.GaussianProcess(
    mean=linpde_gp.functions.StackedFunction(
        # u
        linpde_gp.functions.Constant(input_shape=(), value=57.0),
        # \dot{q}_V
        cpu.q_dot_V_estim_1D,
        # \dot{q}_A
        linpde_gp.functions.Constant(input_shape=(), value=cpu.q_dot_A_1D[0]),
    ),
    cov=linpde_gp.randprocs.covfuncs.IndependentMultiOutputCovarianceFunction(
        # u
        3.0 ** 2 * linpde_gp.randprocs.covfuncs.Matern(
            input_shape=(),
            nu=2.5,
            lengthscales=0.75 * cpu.width,
        ),
        # \dot{q}_V
        0.9 ** 2 * linpde_gp.randprocs.covfuncs.Matern(
            input_shape=(),
            nu=0.5,
            lengthscales=cpu.width,
        ),
        # \dot{q}_A
        0.9 ** 2 * linpde_gp.randprocs.covfuncs.Matern(
            input_shape=(),
            nu=0.5,
            lengthscales=cpu.width,
        )
    )
)

In [ ]:
select_u = linpde_gp.linfuncops.SelectOutput(input_shapes=((), (3,)), idx=0)
select_q_dot_V = linpde_gp.linfuncops.SelectOutput(input_shapes=((), (3,)), idx=1)
select_q_dot_A = linpde_gp.linfuncops.SelectOutput(input_shapes=((), (3,)), idx=2)

In [ ]:
plot_belief(
    plt.gca(),
    u=select_u(ufg_prior),
)

In [ ]:
plot_pred_belief(
    plt.gca(),
    u=select_u(ufg_prior),
    q_dot_V=select_q_dot_V(ufg_prior),
)

In [ ]:
with plt.rc_context(config.tueplots_bundle(rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower center"}):
        plot_pred_belief(
            ax=plt.gca(),
            q_dot_V=select_q_dot_V(ufg_prior),
        )

experiment_utils.savefig("03_full_00_q_dot_V_prior")

In [ ]:
with plt.rc_context(config.tueplots_bundle(nrows=2, rel_width=0.5)):
    with plt.rc_context({
        "legend.facecolor": "white",
        "legend.framealpha": 0.8,
    }):
        fig, ax = plt.subplots(nrows=2, sharex=True)

        with plt.rc_context({"legend.loc": "lower left"}):
            plot_belief(
                ax=ax[0],
                u=select_u(ufg_prior),
            )

        with plt.rc_context({"legend.loc": "upper right"}):
            plot_pred_belief(
                ax=ax[1],
                bvp=cpu.bvp_1D_dbc,
                # u=select_u(ufg_prior),
                q_dot_V=select_q_dot_V(ufg_prior),
            )

experiment_utils.savefig("03_full_00_prior")

### Observations

In [ ]:
N_pde = 17

X_pde = bvp.domain.uniform_grid(N_pde, inset=0.03 * cpu.width)

In [ ]:
X_dts = cpu.X_dts_1D
y_dts = cpu.y_dts_1D
noise_dts = cpu.noise_dts_1D

### Effect of (Thermal) Stationarity on the Prior

In [ ]:
L_stat = (
    cpu.height * linpde_gp.linfunctls.LebesgueIntegral(input_domain=bvp.domain) @ select_q_dot_V
    + cpu.height * (select_q_dot_A.to_linfunctl(cpu.width) + select_q_dot_A.to_linfunctl(0.0))
)

In [ ]:
ufg_cond_stat = ufg_prior.condition_on_observations(
    Y=0.0,
    L=L_stat,
)

In [ ]:
plot_pred_belief(
    plt.gca(),
    u=select_u(ufg_cond_stat),
    q_dot_V=select_q_dot_V(ufg_cond_stat),
    conditioned_on=["stat"],
)

In [ ]:
with plt.rc_context(config.tueplots_bundle(rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower center"}):
        plot_pred_belief(
            ax=plt.gca(),
            q_dot_V=select_q_dot_V(ufg_cond_stat),
        )

experiment_utils.savefig("03_full_01_q_dot_V_cond_stat")

In [ ]:
with plt.rc_context(config.tueplots_bundle(nrows=2, rel_width=0.5)):
    with plt.rc_context({
        "legend.facecolor": "white",
        "legend.framealpha": 0.8,
    }):
        fig, ax = plt.subplots(nrows=2, sharex=True)

        with plt.rc_context({"legend.loc": "lower left"}):
            plot_belief(
                ax=ax[0],
                u=select_u(ufg_cond_stat),
                # conditioned_on=["stat"],
            )

        with plt.rc_context({"legend.loc": "upper right"}):
            plot_pred_belief(
                ax=ax[1],
                # u=select_u(ufg_prior),
                q_dot_V=select_q_dot_V(ufg_cond_stat),
                # conditioned_on=["stat"],
            )

experiment_utils.savefig("03_full_01_cond_stat")

### Posterior

#### Conditioning on the PDE with Uncertain RHS

In [ ]:
ufg_cond_pde = ufg_prior.condition_on_observations(
    Y=np.zeros_like(X_pde),
    L=bvp.pde.diffop @ select_u - select_q_dot_V,
    X=X_pde,
)

In [ ]:
plot_belief(
    plt.gca(),
    u=select_u(ufg_cond_pde),
    conditioned_on=["pde"],
    X_pde=X_pde,
)

In [ ]:
plot_pred_belief(
    plt.gca(),
    u=select_u(ufg_cond_pde),
    q_dot_V=select_q_dot_V(ufg_cond_pde),
    conditioned_on=["pde"],
    X_pde=X_pde,
)

In [ ]:
with plt.rc_context(config.tueplots_bundle(nrows=2, rel_width=0.5)):
    with plt.rc_context({
        "legend.facecolor": "white",
        "legend.framealpha": 0.8,
    }):
        fig, ax = plt.subplots(nrows=2, sharex=True)

        with plt.rc_context({"legend.loc": "lower left"}):
            plot_belief(
                ax=ax[0],
                u=select_u(ufg_cond_pde),
                # conditioned_on=["pde"],
                X_pde=X_pde,
                show_u_label=False,
            )

        with plt.rc_context({"legend.loc": "upper right"}):
            plot_pred_belief(
                ax=ax[1],
                # u=select_u(ufg_cond_pde),
                q_dot_V=select_q_dot_V(ufg_cond_pde),
                # conditioned_on=["pde"],
                X_pde=X_pde,
                show_u_f_label=False,
            )

experiment_utils.savefig("03_full_01_cond_pde")

#### Conditioning on Uncertain Neumann Boundary Conditions

In [ ]:
ufg_cond_pde_nbc = ufg_cond_pde

for idx, bcond in enumerate(bvp.boundary_conditions):
    ufg_cond_pde_nbc = ufg_cond_pde_nbc.condition_on_observations(
        Y=0.0,
        L=bcond.operator @ select_u - select_q_dot_A,
        X=bcond.boundary,
    )

In [ ]:
plot_belief(
    plt.gca(),
    u=select_u(ufg_cond_pde_nbc),
    conditioned_on=["pde", "nbc"],
    X_pde=X_pde,
    q_dot_A=select_q_dot_A(ufg_cond_pde_nbc)([0.0, cpu.width]),
)

In [ ]:
plot_pred_belief(
    plt.gca(),
    u=select_u(ufg_cond_pde_nbc),
    q_dot_V=select_q_dot_V(ufg_cond_pde_nbc),
    conditioned_on=["pde", "nbc"],
    X_pde=X_pde,
)

In [ ]:
with plt.rc_context(config.tueplots_bundle(nrows=2, rel_width=0.5)):
    with plt.rc_context({
        "legend.facecolor": "white",
        "legend.framealpha": 0.8,
    }):
        fig, ax = plt.subplots(nrows=2, sharex=True)

        with plt.rc_context({"legend.loc": "lower left"}):
            plot_belief(
                ax=ax[0],
                u=select_u(ufg_cond_pde),
                # conditioned_on=["pde", "nbc"],
                X_pde=X_pde,
                q_dot_A=select_q_dot_A(ufg_cond_pde_nbc)([0.0, cpu.width]),
                show_u_label=False,
            )

        with plt.rc_context({"legend.loc": "upper right"}):
            plot_pred_belief(
                ax=ax[1],
                # u=select_u(ufg_cond_pde),
                q_dot_V=select_q_dot_V(ufg_cond_pde),
                # conditioned_on=["pde", "nbc"],
                X_pde=X_pde,
                show_u_f_label=False,
            )

experiment_utils.savefig("03_full_02_cond_pde_nbc")

### Measurements

In [ ]:
ufg_cond_pde_nbc_dts = ufg_cond_pde_nbc.condition_on_observations(
    Y=y_dts,
    L=select_u,
    X=X_dts,
    b=noise_dts,
)

In [ ]:
with plt.rc_context(config.tueplots_bundle(nrows=2, rel_width=0.5)):
    with plt.rc_context({
        "legend.facecolor": "white",
        "legend.framealpha": 0.8,
    }):
        fig, ax = plt.subplots(nrows=2, sharex=True)

        with plt.rc_context({"legend.loc": "lower left"}):
            plot_belief(
                ax=ax[0],
                u=select_u(ufg_cond_pde_nbc_dts),
                # conditioned_on=["pde", "nbc", "dts"],
                X_pde=X_pde,
                q_dot_A=select_q_dot_A(ufg_cond_pde_nbc_dts)([0.0, cpu.width]),
                X_dts=X_dts,
                y_dts=y_dts,
                noise_dts=noise_dts,
                show_u_label=False,
            )

        with plt.rc_context({"legend.loc": "upper right"}):
            plot_pred_belief(
                ax=ax[1],
                # u=select_u(ufg_cond_stat_pde_nbc_dts),
                q_dot_V=select_q_dot_V(ufg_cond_pde_nbc_dts),
                # conditioned_on=["pde", "nbc", "dts"],
                X_pde=X_pde,
                show_u_f_label=False,
            )

experiment_utils.savefig("03_full_03_cond_pde_nbc_dts")

#### Conditioning on (Thermal) Stationarity

In [ ]:
ufg_cond_pde_nbc_dts_stat = ufg_cond_pde_nbc_dts.condition_on_observations(
    Y=0.0,
    L=L_stat,
)

In [ ]:
with plt.rc_context(config.tueplots_bundle(rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower center"}):
        plot_belief(
            plt.gca(),
            u=select_u(ufg_cond_pde_nbc_dts_stat),
            conditioned_on=["pde", "nbc", "dts", "stat"],
            X_pde=X_pde,
            q_dot_A=select_q_dot_A(ufg_cond_pde_nbc_dts_stat)([0.0, cpu.width]),
            X_dts=X_dts,
            y_dts=y_dts,
            noise_dts=noise_dts,
        )

experiment_utils.savefig("03_full_03_cond_pde_nbc_stat_dts_u")

In [ ]:
with plt.rc_context(config.tueplots_bundle(rel_width=0.5)):
    with plt.rc_context({"legend.loc": "lower center"}):
        plot_pred_belief(
            plt.gca(),
            q_dot_V=select_q_dot_V(ufg_cond_pde_nbc_dts_stat),
            conditioned_on=["pde", "nbc", "dts", "stat"],
        )

experiment_utils.savefig("03_full_03_cond_pde_nbc_stat_dts_f")

In [ ]:
with plt.rc_context(config.tueplots_bundle(nrows=2, rel_width=0.5)):
    with plt.rc_context({
        "legend.facecolor": "white",
        "legend.framealpha": 0.8,
    }):
        fig, ax = plt.subplots(nrows=2, sharex=True)

        with plt.rc_context({"legend.loc": "lower left"}):
            plot_belief(
                ax=ax[0],
                u=select_u(ufg_cond_pde_nbc_dts_stat),
                # conditioned_on=["pde", "nbc", "dts", "stat"],
                X_pde=X_pde,
                q_dot_A=select_q_dot_A(ufg_cond_pde_nbc_dts_stat)([0.0, cpu.width]),
                X_dts=X_dts,
                y_dts=y_dts,
                noise_dts=noise_dts,
                show_u_label=False,
            )

        with plt.rc_context({"legend.loc": "upper right"}):
            plot_pred_belief(
                ax=ax[1],
                # u=select_u(ufg_cond_stat_pde_nbc_dts),
                q_dot_V=select_q_dot_V(ufg_cond_pde_nbc_dts_stat),
                # conditioned_on=["pde", "nbc", "dts"],
                X_pde=X_pde,
                show_u_f_label=False,
            )

experiment_utils.savefig("03_full_04_cond_pde_nbc_dts_stat")